In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import urllib.request as request
from urllib.parse import quote

import ruptures as rpt

from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

In [3]:
def get_token():
    refresh_token = 'eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ0b2tlblR5cGUiOiJUSElSRFBBUlRZQVBJX1JlZnJlc2giLCJ0b2tlbmlkIjoiMTU3ZDY4N2QtZDBjZS00OTcyLWI5MjQtMzJhMzY0ODFkYjYxIiwid2ViQXBwIjoiVGhpcmRQYXJ0eUFwcCIsInZlcnNpb24iOiIyIiwiaWRlbnRpdHlUb2tlbiI6Ik1sdDYzOTg2VS8vaUZ4aFhPSCthczVFbVh0U1FPUVBDTXorY1VqejdFSzNPTWx5VGVUbUdSUm9WZDZYa0RsMkR3S1NnSlVxTWZaejVHVjRYZ1F0Mzl5WnNoWEQvZUg3MjFkRzlzQzJDV2dSZC9STFNZZjNvSmhTRXdnRTdJbFZkMkRaSVY0SDlPTUpwOGh1VWtpK1VweFZIUDNwL09TTkhKVFJIekgyOE9CYlhwZmJQSEYveFd1NGhkK1p3RWRPSU8zZUFSK2EvVDFUcDc3cHFjOTU3aWxlb3plWDJmKzVtUk8xMnJaUTBIUG4wZFNTMzVlSEx0VVRwRG0vNmVCSDRNZUY5MENDaXMrc1ZoZDRJSE85ZWVyMkRPbGF1U2I5SVFkQk5XdDBQNEJoS3dOcWg0Y3lFMGZ4bWUvaDNXS1NYRVRDL0treWFaYSszNC9YVS94cWJ6a0J2NWhXSjhkYVZ2YTJlTE1jSHZpUkszRXRoMHVwMmxUT0lKZEpFWkJrL1JvcWhQelR2anVUNTZZZGEwZlp5VG5uYno5NkR0eVlqZC9tNlpuNDNzdDRJUmZFbVpzOWg0STU2b0ZoSkR4d0hFdnhra2IrMHhpYW1UMllhVjJyMTRKeDJ0dE1QR2N5S1RscllkVWE2a2pHYXlzRHI4aHhVdnlIYnBUeXFvd3V0bmZpYTQ2TnZOOVo3cUt1czhkK3E5YkgrV280RkQ2Ym1OMzMramRUWUFUY21TTWJya0kzWm9GSmxhYUFpWEJlMkdzRTNiVnFkcmxoTGtGTUtINFp5MGNuMTNzK21BTkd5eTZDcUdlQ2YweXVrYjJYUGpBb1oyNEdNN2dESjVKVmZjc2dPWDVMcURmMUZsUVdRbzRvd1pzSXBUd3ZMTWZXRXI5UnlETUJ3SkRZMnUwMWw2SmxQbllReU5ZRDd2ZkRtQjYwMVNaUm9CazBrMmxJMUp4SEk4UFRWVjNYT2Q1NWFaZDVXajJRMkdZcEd0NmlmNDNRZkV5aTMvd1k1QmpSMk1qNzg3NUhUUi9Xc2pqQlY5U0QwQmZ1ODVwT0J5WHVCenBRRmlCWHpzL0Z3cm9McWE4RHYyaTZxZEZ1UGZOM2QvSVJ3bmErRTlyQUdJRXZNZHV0dmREZE9nTEhsYm04NlE5aUU0NkhGcWdyd2JXY0tzMkdaK3dDSnhKclZaT2ZMSVQvQVVQcGNaKzdGbDdFNDNPT2dMSzdycEhFK2VJRitGRWZGeDAwMGVMekpERWdaTGdGRVVWMFFsRGtVZjR3bGtxNW1RaHNMNG5ualphb200dStzYVBraW5sV2NCTU5QdVFETzFZVFlhZTFZQmFKNkcyelo3OWU4aDhvQmQzM1RtMXNDSUorbTcwNXNjNm9nNUoyaEd1LzRtaXFEeE9pbjQ2ek5tS0djLzFqejlwT1I4MktrMXgxOXhWYm4vOXpKU1hnUGZSbEloZ2hQTVQrb3dEUXpaT2xJdWx4SVAxSGpib2I3TmhZRFl6T1ArMkdJU3JxS2k4dFBadXpYRnFZQ3RZN3RaMEFYTXRNQWVHbnhFNi9JNW9iKzI0VDVzRUN0S3c4T2RTb3ZJQlNCeU45R3YyczY0ZTNOWHBMNWZGcG80VGtoanZxYnhMNVhqR3FsZXc0cktjbDZmdFdRTTlGaTBXTEFERVZJek1McHpZRWhLSEtIQytHZDhidlI5UmI3SkNTZ0tySUVwZ1FyV2ZpVWcxazdQb3ZENVZCR04yT2l6dTR1Y1dkUmh1VDBkQTRQcXEyV0NTSCtYcVNzZHNnaWgxKzQ5c3pkTUJVck1BOGdvNENraDRjdVRNbmY4NTdWamVBWTV2aGp6NjM0ay9mOGxzZ2pxdGs3WHlOUUJjSk0ycU0vRjNrODY5OEt5V2l1NjM4RHhkMS9wV1lYdWEzQ0VtdlV2SlNxVUFpOGVHV1B1VngzcG9qeEFRNTVXZTgwSFUrbk93U01tYUJEZ0dKbGxsaFh6alZkNUd0TTNhcnYrNWRad3FaQWxVdWtwVVZYKy92ekg1MFNPU1M2c2Y4OTdwclNheHprQnpzdlFxWFpmTDFRc2I3dERMZDloa1R2cHZRbXl6d1ZJVlU9IiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvbmFtZWlkZW50aWZpZXIiOiJDVlI6MzMwNzc4MzEtUklEOjY0OTI5MzMxIiwiaHR0cDovL3NjaGVtYXMueG1sc29hcC5vcmcvd3MvMjAwNS8wNS9pZGVudGl0eS9jbGFpbXMvZ2l2ZW5uYW1lIjoiTWlra2VsIFJhc211cyBTY2htaWR0IiwibG9naW5UeXBlIjoiQ2VydGlmaWNhdGUiLCJyaWQiOiI2NDkyOTMzMSIsImN2ciI6IjMzMDc3ODMxIiwiY29tcGFueSI6Ik5SR2kgUsOlZGdpdm5pbmcgQS9TICIsInVzZXJJZCI6IjY5ODkwMSIsInRwaWQiOiIzQUI2MDJBRC03OEIxLTRFMDItOEUxNi1FMzI2NDIwNDAzRDgiLCJleHAiOjE3MTM0MjQ2MjksImlzcyI6IkVuZXJnaW5ldCIsImp0aSI6IjE1N2Q2ODdkLWQwY2UtNDk3Mi1iOTI0LTMyYTM2NDgxZGI2MSIsInRva2VuTmFtZSI6Ik1pa2tlbCIsInJvbGVzIjoiUmVhZFByaXZhdGUsIFJlYWRCdXNpbmVzcyIsImF1ZCI6IkVuZXJnaW5ldCJ9.VoP6PRoM1GFi8XPYzIjppo66fv-NrUXBq1ooJx3HdPc'
    try:
        access_token
    except NameError:
        url = "https://api.eloverblik.dk/ThirdPartyApi" + '/api/Token'
        headers = {'Authorization': 'Bearer ' + refresh_token}
        token_response = requests.get(url, headers=headers, timeout=5)
        token_response.raise_for_status()
        token_json = token_response.json()
        access_token = token_json['result']
        timelog = datetime.now() - timedelta(days=2)
    else:
        time_between_insertion = datetime.now() - timelog
        if  time_between_insertion.days > 1:
            url = "https://api.eloverblik.dk/ThirdPartyApi" + '/api/Token'
            headers = {'Authorization': 'Bearer ' + refresh_token}
            token_response = requests.get(url, headers=headers, timeout=5)
            token_response.raise_for_status()
            token_json = token_response.json()
            access_token = token_json['result']
            
            timelog = datetime.now()
    return access_token

def getMeterPoints(Customer):
    ApiKey = "3bb0bb5c-392d-4daa-8b0e-6a675387d08b"
    url = "https://superhub.dk/api/nrgiraadgivning/v2/meteringPoints"
    url = url + "?" + "apikey=" + ApiKey + "&customerKey=" + quote(Customer)
    #print(url)
    response = requests.get(url)
    df = pd.json_normalize(response.json()) 
    df['meteringPointId'] = pd.to_numeric(df['meteringPointId'])
    #print(df['meteringPointId'].to_list())
    IDs = df['meteringPointId'].to_list()
    return df, IDs



def getMeterReadings(meteringPoints):
    ApiKey = "3bb0bb5c-392d-4daa-8b0e-6a675387d08b"
    From = '31122021'#(datetime.today() - timedelta(days = 1000)).strftime('%d%m%Y')
    To = datetime.today().strftime('%d%m%Y')
    df = pd.DataFrame()
    Fails = 0
    for meter in tqdm(meteringPoints):
        url = "https://superhub.dk/api/nrgiraadgivning/v2/meterreadings"
        url = url + "?" + "apikey=" + ApiKey + "&meteringpointId=" + str(meter) + "&from=" + From + "&to=" + To
        
        response = requests.get(url)
        if response.status_code != 200:
            Fails += 1
            continue
        jso = response.json()

        dff = pd.json_normalize(jso)
        
        if (dff.columns == 'meteringPoints.Production').any():
            df_meter = pd.json_normalize(jso['meteringPoints'], 'Production')
        elif (dff.columns == 'meteringPoints.Consumption').any():
            df_meter = pd.json_normalize(jso['meteringPoints'], 'Consumption') 
        else:
             print(dff.columns)
             continue
        df_meter['meter'] = jso['meteringPointId']
        df_meter['Adresse'] = jso['streetName'] + ' ' + jso['buildingNumber'] + ', ' + jso['postcode'] + ' ' + jso['cityName']
        df = pd.concat([df, df_meter], ignore_index=True)
    df['from'] = pd.to_datetime(df['from'], utc=True)
    df['meter'] = pd.to_numeric(df['meter'])
    print('Amount of fails: ' + str(Fails))
    print(url)
    return df

def get_day_moment(hour) -> str: 
    if 6 <= hour <= 18:
        return 'day'
    return 'night'

def ugeprofil(df):
            dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
            dff['day_'] = dff['day']
            dff['day_'].replace({
                    "Mandag": 0,
                    "Tirsdag": 1,
                    "Onsdag": 2,
                    "Torsdag": 3,
                    "Fredag": 4,
                    "Lørdag": 5,
                    "Søndag": 6},
                    inplace=True,)
            dff.sort_values(['day_', 'hour'], ascending=True, inplace=True)
            dff['x-axis'] = dff.apply(lambda row: row['day'] + ' kl. ' + str(row['hour']), axis=1)
            return dff

def rupt(tid, df):
    nbkps = 12

    test = df['amount']
    points=np.array(test)
    n= len(points)
    if nbkps == 1:
        nbkps = np.floor(np.round(n/365*4))
    #Changepoint detection with dynamic programming search method
    model = "l1"  
    algo = rpt.Window(width=24*30, model=model, min_size=3, jump=5).fit(points)
    my_bkps = algo.predict(n_bkps=nbkps)
    return my_bkps

def besp():
    df_besp = pd.DataFrame(columns=['Adresse', 'besparelse', 'årligt forbrug', 'last', 'best', 'mean', 'bkps'])
    dfff = pd.DataFrame()
    
    for adr in tqdm(df['Adresse'].unique()):
        dff = df[df['Adresse']==adr]
        dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
        dff = dff.groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first', 'Adresse': 'first'}).reset_index()
        my_bkps = rupt('day', dff)

        value_avg_day = np.zeros(len(dff['amount']))
        #stdd_avg_day  = np.zeros(len(dff['amount']))
        vvvv = np.zeros((len(my_bkps),2),)
        k=0
        j=0
        for i in my_bkps:
            value_avg_day[j:i] = np.mean(dff['amount'][j:i])
            #stdd_avg_day[j:i]  = np.std(dff['amount'][j:i])
            vvvv[k, 0] = np.mean(dff['amount'][j:i])
            vvvv[k, 1] = i
            k += 1
            j=i
        dff['bkps'] = value_avg_day
        print(dff['bkps'])
        
        if dff['bkps'].iloc[-1] >= dff['bkps'].max():
            df_opti = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
        else:
            df_opti = dff[dff['bkps']==dff['bkps'].min()].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
            k=1
            while df_opti['amount'].mean() <= 0.2:
                dff1 = dff[~(dff['bkps']==dff['bkps'].min())]
                df_opti = dff1[dff1['bkps']==dff1['bkps'].min()].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
                k += 1
                del dff1
                if (k == 10):
                    df_opti = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()
                    break
                
        df_norm = dff[dff['bkps']==dff['bkps'].iloc[-1]].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'day-moment': 'first'}).reset_index()

        uge = ugeprofil(df_opti)
        uge2 = ugeprofil(df_norm) 
        #print(my_bkps)

        ugg = uge[['day', 'hour', 'amount', 'x-axis']].merge(uge2[['day', 'hour', 'amount']], how='outer', on=['day', 'hour'], suffixes=('_opti', '_now'))
        ugg['besparelse_kwh'] = ugg['amount_now'] - ugg['amount_opti']
        ttt = pd.DataFrame(data={'Adresse': [adr], 
                                 'besparelse': [ugg['besparelse_kwh'].sum()*52], 
                                 'årligt forbrug': [ugg['amount_now'].sum()*52],
                                 'last': df_norm['amount'].mean(),
                                 'best': df_opti['amount'].mean(), 
                                 'mean': dff['amount'].mean(),
                                 'meter': dff['meter'].mean(),
                                 'bkps': 0,
                                 'bkps_value': 0,
                                 'total kwh': dff['amount'].sum()} )
        ttt[['bkps', 'bkps_value']] = ttt[['bkps', 'bkps_value']].astype('object')
        ttt.at[0,'bkps'] = my_bkps
        ttt.at[0,'bkps_value'] = vvvv[:,0]
        df_besp = df_besp.append(ttt)
        dfff = dfff.append(dff)

    df_besp = df_besp[df_besp['årligt forbrug'] != 0.0]
    df_besp['%'] = df_besp.apply(lambda row: row['besparelse']/row['årligt forbrug']*100, axis=1)
    return df_besp, dfff

In [20]:
data = pd.read_csv('../MeteringPoints.csv', sep=';')
data2 = pd.read_csv('../Meterdata.csv', sep=';')
data = data.merge(data2)
data = data[['MålepunktsID', 'Juridisk_kontakt_By', 'Fra_dato', 'Til_dato', 'Mængde']]
data.columns = ['meter', 'Adresse', 'from', 'to', 'amount']
data['from'] = pd.to_datetime(data['from'])
#data['amount'] = pd.to_numeric(data['amount'])
data['day-moment'] = data['from'].dt.hour.map(get_day_moment)

data.head()

,meter,Adresse,from,to,amount,day-moment
0,571313123200330480,Mårslet,2019-04-19 00:00:00,19-04-2019 01:00,0.08,night
1,571313123200330480,Mårslet,2019-04-19 01:00:00,19-04-2019 02:00,0.07,night
2,571313123200330480,Mårslet,2019-04-19 02:00:00,19-04-2019 03:00,0.06,night
3,571313123200330480,Mårslet,2019-04-19 03:00:00,19-04-2019 04:00,0.07,night
4,571313123200330480,Mårslet,2019-04-19 04:00:00,19-04-2019 05:00,0.05,night


In [21]:
df = data
df_besp, dff = besp()
dff['meter'] = pd.to_numeric(dff['meter'])
kunde = 'Privat_elforbrug_test'
df_besp.head()
dff = dff.iloc[889:]
dff.head()


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_26464\2808280486.py:169: FutureWarning: The frame.append method is

0        0.969780
1        0.969780
2        0.969780
3        0.969780
4        0.969780
           ...   
35032    0.583188
35033    0.583188
35034    0.583188
35035    0.583188
35036    0.583188
Name: bkps, Length: 35037, dtype: float64


,from,meter,amount,day-moment,Adresse,bkps
889,2019-04-24 00:00:00,5.713131e+17,0.08,night,Mårslet,0.96978
890,2019-04-24 01:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
891,2019-04-24 02:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
892,2019-04-24 03:00:00,5.713131e+17,0.07,night,Mårslet,0.96978
893,2019-04-24 04:00:00,5.713131e+17,0.10,night,Mårslet,0.96978


In [19]:
kunde = 'Privat_elforbrug_test'
df, IDs = getMeterPoints(kunde)
print(IDs)
df.head()

[571313123200330480]


,meteringPointId,streetName,buildingNumber,postcode,cityName
0,571313123200330480,Nymarksvej,14C,8320,Mårslet


In [8]:
df = getMeterReadings(IDs)

100%|██████████| 14/14 [00:07<00:00,  1.93it/s]


Amount of fails: 0
https://superhub.dk/api/nrgiraadgivning/v2/meterreadings?apikey=3bb0bb5c-392d-4daa-8b0e-6a675387d08b&meteringpointId=571313174115543007&from=31122021&to=19042023


In [15]:
df.head()

,from,to,amount,unit,meter,Adresse,day-moment
0,2021-12-31 23:00:00+00:00,2022-01-01T01:00:00+01:00,0.77,KWH,571313105202834234,"Troensevej 4B, 9220 Aalborg Øst",night
1,2022-01-01 00:00:00+00:00,2022-01-01T02:00:00+01:00,0.79,KWH,571313105202834234,"Troensevej 4B, 9220 Aalborg Øst",night
2,2022-01-01 01:00:00+00:00,2022-01-01T03:00:00+01:00,0.81,KWH,571313105202834234,"Troensevej 4B, 9220 Aalborg Øst",night
3,2022-01-01 02:00:00+00:00,2022-01-01T04:00:00+01:00,0.77,KWH,571313105202834234,"Troensevej 4B, 9220 Aalborg Øst",night
4,2022-01-01 03:00:00+00:00,2022-01-01T05:00:00+01:00,0.76,KWH,571313105202834234,"Troensevej 4B, 9220 Aalborg Øst",night


In [9]:
df['day-moment'] = df['from'].dt.hour.map(get_day_moment)

In [35]:
df_besp, dff = besp()
dff['meter'] = pd.to_numeric(dff['meter'])
df_besp.head()

  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only i

0        1.441511
1        1.441511
2        1.441511
3        1.441511
4        1.441511
           ...   
11082    1.082458
11083    1.082458
11084    1.082458
11085    1.082458
11086    1.082458
Name: bkps, Length: 11087, dtype: float64


C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated.

0        2.897887
1        2.897887
2        2.897887
3        2.897887
4        2.897887
           ...   
11082    2.359379
11083    2.359379
11084    2.359379
11085    2.359379
11086    2.359379
Name: bkps, Length: 11087, dtype: float64


C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated.

...

C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated.

0        0.909707
1        0.909707
2        0.909707
3        0.909707
4        0.909707
           ...   
11130    0.499399
11131    0.499399
11132    0.499399
11133    0.499399
11134    0.499399
Name: bkps, Length: 11135, dtype: float64


C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['day-moment'] = dff['from'].dt.hour.map(get_day_moment)
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dff = df.groupby([df['from'].dt.day_name(locale='da_DK'), df['from'].dt.hour]).mean().reset_index(names=['day', 'hour'])
C:\Users\misc\AppData\Local\Temp\ipykernel_33608\2808280486.py:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated.

0        2.055019
1        2.055019
2        2.055019
3        2.055019
4        2.055019
           ...   
11130    1.327800
11131    1.327800
11132    1.327800
11133    1.327800
11134    1.327800
Name: bkps, Length: 11135, dtype: float64


,Adresse,besparelse,årligt forbrug,last,best,mean,bkps,meter,bkps_value,total kwh,%
0,"Troensevej 4B, 9220 Aalborg Øst",820.826165,9444.228831,1.082458,0.973691,1.130718,"[675, 1210, 2185, 3020, 3765, 4770, 6540, 8060...",5.713131e+17,"[1.4415111111111112, 1.3358691588785048, 1.424...",12536.27,8.691299
0,"Troensevej 4A, 9220 Aalborg Øst",10491.676000,20614.048000,2.359379,1.142112,2.036900,"[975, 2550, 3220, 3705, 4200, 5090, 6680, 7780...",5.713131e+17,"[2.8978871794871797, 2.6865269841269845, 2.061...",22583.11,50.895758
0,"Tangen 8, 8200 Aarhus N",16594.254857,534529.542857,60.936778,59.070990,66.642585,"[4075, 6040, 7720, 8690, 10055, 11135]",5.713131e+17,"[67.35775214723925, 73.28201526717558, 67.5437...",742065.18,3.104460
0,"Mirabellevej 11F, 8930 Randers NØ",0.000000,1182.688788,0.135199,0.135199,0.153759,"[510, 1640, 2010, 3305, 4875, 5275, 7630, 9200...",5.713131e+17,"[0.22660784313725488, 0.13516814159292037, 0.4...",1712.11,0.000000
0,"Saturnvej 3, 6000 Kolding",0.000000,107171.566667,12.155036,12.155036,15.653922,"[1425, 3075, 5260, 6510, 7725, 9545, 10720, 11...",5.713131e+17,"[13.909789473684212, 13.473842424242422, 19.46...",174306.42,0.000000


In [22]:
df_besp.to_csv('../Data/besp/'+kunde+'.csv', header=True, sep=',', index=False)
dff.to_csv('../Data/timeforbrug/'+kunde+'.csv', header=True, sep=',', index=False) 

In [ ]:
dff.shape  

In [ ]:
#df_besp.head()

In [ ]:
dfff = dff[dff['Adresse']=='Vandmanden 5, 9200 Aalborg SV'].groupby('from').agg({'meter': 'mean', 'amount': 'sum', 'bkps': 'sum'}).reset_index()

fig, ax = plt.subplots(figsize=(14,4)) 
ax.plot(dfff['from'], dfff['amount'], linewidth=0.3)
ax.plot(dfff['from'], dfff['bkps'])
ax.plot(dfff['from'][dfff['bkps']==dfff['bkps'].min()], dfff['bkps'][dfff['bkps']==dfff['bkps'].min()], linewidth=6)


In [ ]:
dfff.head()